## Embedding

In [1]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [3]:
# 단어 사전 구축 및 단어별 인덱스 부여, 정수화
dics = {}
for word in nouns: # ['오늘', '날씨', '구름']
    if word not in dics.keys():
        print('dics.keys():', dics.keys())
        print('len(dics):', len(dics))
        # dics dictionary에 word란 키로 dictionary 길이 할당
        dics[word] = len(dics) # dics['오늘'] = 0
        
print(dics)
print(list(dics.values())) # 값만 출력

dics.keys(): dict_keys([])
len(dics): 0
dics.keys(): dict_keys(['오늘'])
len(dics): 1
dics.keys(): dict_keys(['오늘', '날씨'])
len(dics): 2
{'오늘': 0, '날씨': 1, '구름': 2}
[0, 1, 2]


In [8]:
# 원-핫 인코딩
nb_classes = len(dics)        # dictionary 길이
print(type(np.eye(nb_classes)))
print(np.eye(nb_classes))

<class 'numpy.ndarray'>
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
targets = list(dics.values()) # [0, 1, 2]
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)
print()
one_hot_targets = np.eye(nb_classes)[[2, 1, 1]]
print(one_hot_targets)

### Word2Vec

In [15]:
line = "8112052	어릴때보고 지금다시봐도 재밌어요ㅋㅋ	1"
print(line.split('\t'))
line = "8132799	디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.	1"
print(line.split('\t'))

['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1']
['8132799', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.', '1']


In [16]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data

In [19]:
# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./data/ratings.txt')
print('2) 리뷰 데이터 전체 개수:', len(review_data)) # 리뷰 데이터 전체 개수
print('3) 말뭉치 데이터 읽기 완료: ', time.time() - start)
print(review_data[0:5])

1) 말뭉치 데이터 읽기 시작
2) 리뷰 데이터 전체 개수: 200000
3) 말뭉치 데이터 읽기 완료:  0.6547574996948242
[['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1'], ['8132799', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.', '1'], ['4655635', '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.', '1'], ['9251303', '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지', '1'], ['10067386', '안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.', '1']]


In [22]:
print([sentence[0] for sentence in review_data[0:2]])
print([sentence[1] for sentence in review_data[0:2]])

['8112052', '8132799']
['어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.']


In [ ]:
start = time.time()

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
# sentence[1]: document, 리뷰, 시간이 많이 필요함 core 6: 208초, i5 2320: 257
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)